In [ ]:
from mongoConnection import MongoConnection
import pandas as pd
import pprint


db = MongoConnection()
ma_list = [10,50]
max_drawdown = 0
win_loss = 0
exp_profit = 0 


In [ ]:
 #This returns an iterable 'cursor' object
res = db.query_from_to(1638331200,1641196800)

df = pd.DataFrame(res)
df.rename(columns = {'x':'Time'}, inplace = True)
split = pd.DataFrame(df["y"].tolist(),columns=["Open","High","Low","Close"])
df = pd.concat([df,split], axis=1)
df = df.drop("y", axis=1)

print(df)



In [ ]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

def max_draw(row):
    local_max = 0
    max_drawdown = 0
    for num in row:
        if num < 0:
            local_max+=num
            if local_max < max_drawdown:
                max_drawdown = local_max
        else:
            local_max = 0
    return max_drawdown

def winCount(row):
    count = 0
    for num in row:
        if num >= 0:
            count+=1
    return count
            
    
            

In [ ]:
# These are the fast and slow moving averages that we are applying

df_ma = df.copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.Close.rolling(window=ma).mean() #Create the moving average column
    
df_ma['DIFF'] = df_ma.MA_10 - df_ma.MA_50 # create a new column that takes the different between the two moving averages
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1) # creates a new column that shifts down the diff column, used to determine if there will be a trade
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1) #Lets you know if you should've made trade based on strategy.

df_trades = df_ma[df_ma.IS_TRADE!=0].copy() # Creates a dataframe to analyzes all entries that you should've traded... for later stats 

df_ma.tail()


In [ ]:

df_trades["DELTA"] = df_trades.Close.diff().shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]
exp_profit = df_trades["GAIN"].sum() # tested... works

max_drawdown = max_draw(df_trades["GAIN"]) #tested... works 

print(exp_profit)
print(max_drawdown)

win_loss = winCount(df_trades["GAIN"]) / (len(df_trades)-1)
print(win_loss)

print(df_trades)